In [3]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt

# Data Exploratory

In [4]:
df = pd.read_csv('stock fundamental data.csv')
len(df)


FileNotFoundError: [Errno 2] No such file or directory: 'stock fundamental data.csv'

In [ ]:
df.head()


In [ ]:
print("The shape of the data is:", df.shape)
print(df.info)

In [ ]:
#find the ticker num 
tic_num = df['tic'].unique()
print("The number of tickers are:", len(tic_num), "tickers") 

In [ ]:
#change columns names 
df = df.rename(columns={'act': 'Total Current Assets', 'ap': 'Accounts Payable', 'ceq': 'Total Ordinary Equity',
                        "cogs": "Cost of Goods Sold", "dlc":"Total Debt in Current Liabilities", "dlcch": "Current Debt Changes", 
                        "dltt": "Total Long-Term Debt", 'dp':'Depreciation and Amortization',
                        'dvc':'Dividends Common/Ordinary', 'ib':'Income Before Extraordinary Items', 
                        'intan': 'Total Intangible Assets','invt': 'Total Inventories','ivao':'Investments and Advances',
                        'lct':'Total Current Liabilities', 'lt':'Total Liabilities', 
                        'nopio':'Nonoperating Income (expense)', 'ppent':'Total Property, Plant&Equipment',
                        'spi':'ecial Items', 'txp':'Income Taxes Payable', 'txt':'Total Income Taxes', 
                        'xad':'Advertising Expense', 'xido':'Extraordinary Items and Discontinued Operations', 
                        'xint':'Total Interest and Related Expense', 'xrd':'Research and Development Expense', 
                        'xsga':'Selling,General and Administrative Expense', 'rect':'Total Receivables', 'at':'Total Assets', 'che':'Cash and Short-Term Investments','mkvalt':'Market Values'})
df.head()

In [ ]:
#check for null data 
df.isnull().sum()

In [ ]:
#df drop the null value 
df = df.ffill(limit=1)
df = df.fillna(0)

In [ ]:
#Evaluate the company financially stability, through out the years. Using: Total Current Assets/Total Current Liabilities 
#classifield stable if > 1 and unstable if <1 
df['ratio'] = df['Total Current Assets']/df['Total Current Liabilities']

#create a new columsn to output fin stables 
#if fi_stable > 1:
df['good stable status'] = (df['ratio'] > 1)
df.head()


In [ ]:
 ###THIS BLOCK OF CODE IS CREDITED TO YAAKOV MY TEAMMATE
#create column 'future mkvalt' that holds the market value of the next year
condition1 = df['tic'].shift(-1) == df['tic']
condition2 = df['fyear'].shift(-1) == df['fyear'] + 1
df['future_mkvalt'] = np.where(condition1 & condition2, df['mkvalt'].shift(-1), np.nan)

#percent change is the percent change in market value from one year to the next
df['percent_change'] = (df['future_mkvalt']-df['mkvalt'])/df['mkvalt']

#if percent change is greater than 20%, classify as undervalued
df['undervalued'] = (df['percent_change'] > 0.10) 


In [ ]:
#Expense 
df['Expense'] = df['Research and Development Expense'] + df['Selling,General and Administrative Expense'] + df['Total Interest and Related Expense'] + df['Advertising Expense'] + df['Nonoperating Income (expense)']

In [ ]:
#drop unecessary columns 
expense_list = ['Research and Development Expense','Selling,General and Administrative Expense','Total Interest and Related Expense','Advertising Expense','Nonoperating Income (expense)']
df = df.drop(columns = expense_list)
df = df.drop(columns = ["gvkey","Cost of Goods Sold","ecial Items","Extraordinary Items and Discontinued Operations","costat","Income Taxes Payable","Total Income Taxes","curcd"])

In [ ]:
#drop NaN, and finite 
df = df.dropna(subset=["future_mkvalt", "percent_change"], how='all') 

In [ ]:
df = df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(0)

In [ ]:
df.isnull().sum()

In [ ]:
df.head(20) 

In [35]:
from sklearn import preprocessing
d = preprocessing.normalize(df,axis=0)
scaled_df = pd.DataFrame(d, columns=names)
scaled_df.head()

ValueError: could not convert string to float: 'AIR'

In [17]:
sns.pairplot(df)

KeyboardInterrupt: 

Error in callback <function flush_figures at 0x7fde591b4310> (for post_execute):


KeyboardInterrupt: 

In [16]:
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics 
from sklearn.metrics import precision_score

# KNN Classifier 

In [17]:
#get training data and testing data 
X = df.drop(columns=['tic', 'fyear', 'future_mkvalt', 'percent_change', 'undervalued'])
y = df['undervalued']


In [18]:
SEED = 77
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=SEED)

In [20]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
score =  knn.score(X_test, y_test)
print(score)
#classification_report 
print(classification_report(y_test, y_pred))
# classifier got around 62% of classes right
print(precision_score(y_test, y_pred))

0.6211193022531409
              precision    recall  f1-score   support

       False       0.69      0.73      0.71      6068
        True       0.49      0.44      0.46      3563

    accuracy                           0.62      9631
   macro avg       0.59      0.58      0.58      9631
weighted avg       0.61      0.62      0.62      9631

0.48651191969887075


In [24]:
gains = 0
investment = 0
arr = y_test.index
len(arr)
for i in range(len(y_pred)):
    if(y_pred[i]):
        index = arr[i]
        investment = investment + 100
        change = 100  + 100*(df.loc[[index]]['percent_change'].values)
        gains = gains + change

print('Initial Investment: ' + str(investment))
print('Total Return: ' + str(gains))
print('Percent increase: ' + str((gains-investment)/investment))       

Initial Investment: 914100
Total Return: [1258459.28119197]
Percent increase: [0.37671948]


# Naives Bayes

In [21]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
GB = GaussianNB()
GB.fit(X_train, y_train)
y_pred = GB.predict(X_test)
accuracy_score = accuracy_score(y_test,y_pred)
print("accuracy_score: ", accuracy_score)
print(precision_score(y_test, y_pred))

accuracy_score:  0.3911328003322604
0.3741384968821792


In [22]:
#confusion matrix 
cm = confusion_matrix(y_test, y_pred)
print('Confusion matrix\n\n', cm)

Confusion matrix

 [[ 347 5721]
 [ 143 3420]]


In [25]:
gains = 0
investment = 0
arr = y_test.index
len(arr)
for i in range(len(y_pred)):
    if(y_pred[i]):
        index = arr[i]
        investment = investment + 100
        change = 100  + 100*(df.loc[[index]]['percent_change'].values)
        gains = gains + change

print('Initial Investment: ' + str(investment))
print('Total Return: ' + str(gains))
print('Percent increase: ' + str((gains-investment)/investment))       

Initial Investment: 914100
Total Return: [1258459.28119197]
Percent increase: [0.37671948]


In [2]:
#Note:model using knn, 